In [1]:
import torch
import cv2
import os
from torchvision.transforms import Compose, ToTensor, Normalize
from torch.utils.data import DataLoader, Dataset
import numpy as np
import warnings

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning, module="timm")

In [3]:
class ImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        image = cv2.imread(img_path)
        if image is None:
            raise ValueError(f"Failed to read image {img_path}. Please check the file.")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        return image, img_name

def save_depth_map(depth_map, output_path):
    depth_map = depth_map.squeeze().cpu().numpy()
    depth_map = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min() + 1e-10) * 255.0
    depth_map = depth_map.astype(np.uint8)
    cv2.imwrite(output_path, depth_map)


In [4]:
output_root = os.path.join(os.getcwd(), 'Depth_Features')
input_root = r'D:\AAU Internship\Code\CWF-788\IMAGE512x384'
splits = ['train_new', 'validation_new', 'test_new']
os.makedirs(output_root, exist_ok=True)

for split in splits:
    os.makedirs(os.path.join(output_root, split), exist_ok=True)

model = torch.hub.load("intel-isl/MiDaS", "DPT_Large", pretrained=True, trust_repo=True)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

transform = Compose([
    ToTensor(),
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

batch_size = 4
for split in splits:
    input_dir = os.path.join(input_root, split)
    output_dir = os.path.join(output_root, split)
    
    if not os.path.exists(input_dir):
        print(f"Warning: {input_dir} does not exist. Skipping {split}.")
        continue
    
    print(f"Processing {split} images...")
    
    dataset = ImageDataset(input_dir, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
    
    for batch_images, batch_names in dataloader:
        batch_images = batch_images.to(device)
        
        with torch.no_grad():
            depth_maps = model(batch_images)
        
        for i, (depth_map, img_name) in enumerate(zip(depth_maps, batch_names)):
            output_path = os.path.join(output_dir, img_name.replace('.jpg', '_depth.png'))
            save_depth_map(depth_map, output_path)
    
    print(f"Finished processing {split} images. Depth maps saved to {output_dir}")

Using cache found in C:\Users\A/.cache\torch\hub\intel-isl_MiDaS_master


Processing train_new images...
Finished processing train_new images. Depth maps saved to D:\AAU Internship\Code\Depth_Features\train_new
Processing validation_new images...
Finished processing validation_new images. Depth maps saved to D:\AAU Internship\Code\Depth_Features\validation_new
Processing test_new images...
Finished processing test_new images. Depth maps saved to D:\AAU Internship\Code\Depth_Features\test_new
